![Spark](http://spark.apache.org/images/spark-logo-trademark.png)

# Apache Spark for Data Science
## Radek Ostrowski

My short Bio

### What is Spark?

Spark Core, SQL, GraphX, Streaming

### What is ML Pipelines?

http://spark.apache.org


MLlib standardizes APIs for machine learning algorithms to make it easier to combine multiple algorithms into a single pipeline, or workflow. This section covers the key concepts introduced by the Pipelines API, where the pipeline concept is mostly inspired by the scikit-learn project.

DataFrame: This ML API uses DataFrame from Spark SQL as an ML dataset, which can hold a variety of data types. E.g., a DataFrame could have different columns storing text, feature vectors, true labels, and predictions.

Transformer: A Transformer is an algorithm which can transform one DataFrame into another DataFrame. E.g., an ML model is a Transformer which transforms a DataFrame with features into a DataFrame with predictions.

Estimator: An Estimator is an algorithm which can be fit on a DataFrame to produce a Transformer. E.g., a learning algorithm is an Estimator which trains on a DataFrame and produces a model.

Pipeline: A Pipeline chains multiple Transformers and Estimators together to specify an ML workflow.

Parameter: All Transformers and Estimators now share a common API for specifying parameters.

# House Price Prediction

Simple use case based on the Kaggle house price dataset: https://www.kaggle.com/c/house-prices-advanced-regression-techniques

![Houses](https://kaggle2.blob.core.windows.net/competitions/kaggle/5407/media/housesbanner.png)

"Ask a home buyer to describe their dream house, and they probably won't begin with the height of the basement ceiling or the proximity to an east-west railroad. But this playground competition's dataset proves that much more influences price negotiations than the number of bedrooms or a white-picket fence.

With 79 explanatory variables describing (almost) every aspect of residential homes in Ames, Iowa, this competition challenges you to predict the final price of each home."

## Running demo on your laptop

```git clone https://github.com/radek1st/house-price-prediction.git```

```cd house-price-prediction```

```docker run -it -v `pwd`:/home/jovyan/work --rm -p 8888:8888 jupyter/all-spark-notebook```

Then navigate to the url displayed in console, something like:

```http://localhost:8888/?token=46c86a1f120a3d3002308```

## Data Exploration

In [1]:
import org.apache.spark.ml.{Pipeline,PipelineModel}
import org.apache.spark.ml.feature.{VectorAssembler,StringIndexer,IndexToString,VectorIndexer}
import org.apache.spark.ml.feature.PCA
import org.apache.spark.ml.clustering.KMeans
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.ml.regression.{LinearRegression,LinearRegressionModel,RandomForestRegressionModel,RandomForestRegressor}
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._

Add visualisation library

In [2]:
%AddJar -magic file:///home/jovyan/work/spark-kernel-brunel-all-2.3.jar

Starting download from file:///home/jovyan/work/spark-kernel-brunel-all-2.3.jar
Finished download of spark-kernel-brunel-all-2.3.jar


In [3]:
val train = spark.read.option("header", true).option("inferSchema", true).csv("train.csv").cache()
val test = spark.read.option("header", true).option("inferSchema", true).csv("test.csv").cache()

In [4]:
train.count

1460

In [5]:
test.count

1459

In [6]:
test.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- MSSubClass: integer (nullable = true)
 |-- MSZoning: string (nullable = true)
 |-- LotFrontage: string (nullable = true)
 |-- LotArea: integer (nullable = true)
 |-- Street: string (nullable = true)
 |-- Alley: string (nullable = true)
 |-- LotShape: string (nullable = true)
 |-- LandContour: string (nullable = true)
 |-- Utilities: string (nullable = true)
 |-- LotConfig: string (nullable = true)
 |-- LandSlope: string (nullable = true)
 |-- Neighborhood: string (nullable = true)
 |-- Condition1: string (nullable = true)
 |-- Condition2: string (nullable = true)
 |-- BldgType: string (nullable = true)
 |-- HouseStyle: string (nullable = true)
 |-- OverallQual: integer (nullable = true)
 |-- OverallCond: integer (nullable = true)
 |-- YearBuilt: integer (nullable = true)
 |-- YearRemodAdd: integer (nullable = true)
 |-- RoofStyle: string (nullable = true)
 |-- RoofMatl: string (nullable = true)
 |-- Exterior1st: string (nullable = true)
 |--

In [7]:
train.schema.size

81

In [8]:
train.select("id","housestyle","bldgtype","neighborhood","overallqual","yearremodadd","saletype","saleprice").show()

+---+----------+--------+------------+-----------+------------+--------+---------+
| id|housestyle|bldgtype|neighborhood|overallqual|yearremodadd|saletype|saleprice|
+---+----------+--------+------------+-----------+------------+--------+---------+
|  1|    2Story|    1Fam|     CollgCr|          7|        2003|      WD|   208500|
|  2|    1Story|    1Fam|     Veenker|          6|        1976|      WD|   181500|
|  3|    2Story|    1Fam|     CollgCr|          7|        2002|      WD|   223500|
|  4|    2Story|    1Fam|     Crawfor|          7|        1970|      WD|   140000|
|  5|    2Story|    1Fam|     NoRidge|          8|        2000|      WD|   250000|
|  6|    1.5Fin|    1Fam|     Mitchel|          5|        1995|      WD|   143000|
|  7|    1Story|    1Fam|     Somerst|          8|        2005|      WD|   307000|
|  8|    2Story|    1Fam|      NWAmes|          7|        1973|      WD|   200000|
|  9|    1.5Fin|    1Fam|     OldTown|          7|        1950|      WD|   129900|
| 10

In [9]:
val schema = StructType(Array(
    StructField("Id", IntegerType, true),
    StructField("MSSubClass", IntegerType, true),
    StructField("MSZoning", StringType, true),
    StructField("LotFrontage", StringType, true),
    StructField("LotArea", IntegerType, true),
    StructField("Street", StringType, true),
    StructField("Alley", StringType, true),
    StructField("LotShape", StringType, true),
    StructField("LandContour", StringType, true),
    StructField("Utilities", StringType, true),
    StructField("LotConfig", StringType, true),
    StructField("LandSlope", StringType, true),
    StructField("Neighborhood", StringType, true),
    StructField("Condition1", StringType, true),
    StructField("Condition2", StringType, true),
    StructField("BldgType", StringType, true),
    StructField("HouseStyle", StringType, true),
    StructField("OverallQual", IntegerType, true),
    StructField("OverallCond", IntegerType, true),
    StructField("YearBuilt", IntegerType, true),
    StructField("YearRemodAdd", IntegerType, true),
    StructField("RoofStyle", StringType, true),
    StructField("RoofMatl", StringType, true),
    StructField("Exterior1st", StringType, true),
    StructField("Exterior2nd", StringType, true),
    StructField("MasVnrType", StringType, true),
    StructField("MasVnrArea", StringType, true),
    StructField("ExterQual", StringType, true),
    StructField("ExterCond", StringType, true),
    StructField("Foundation", StringType, true),
    StructField("BsmtQual", StringType, true),
    StructField("BsmtCond", StringType, true),
    StructField("BsmtExposure", StringType, true),
    StructField("BsmtFinType1", StringType, true),
    StructField("BsmtFinSF1", StringType, true),
    StructField("BsmtFinType2", StringType, true),
    StructField("BsmtFinSF2", StringType, true),
    StructField("BsmtUnfSF", StringType, true),
    StructField("TotalBsmtSF", StringType, true),
    StructField("Heating", StringType, true),
    StructField("HeatingQC", StringType, true),
    StructField("CentralAir", StringType, true),
    StructField("Electrical", StringType, true),
    StructField("1stFlrSF", IntegerType, true),
    StructField("2ndFlrSF", IntegerType, true),
    StructField("LowQualFinSF", IntegerType, true),
    StructField("GrLivArea", IntegerType, true),
    StructField("BsmtFullBath", StringType, true),
    StructField("BsmtHalfBath", StringType, true),
    StructField("FullBath", IntegerType, true),
    StructField("HalfBath", IntegerType, true),
    StructField("BedroomAbvGr", IntegerType, true),
    StructField("KitchenAbvGr", IntegerType, true),
    StructField("KitchenQual", StringType, true),
    StructField("TotRmsAbvGrd", IntegerType, true),
    StructField("Functional", StringType, true),
    StructField("Fireplaces", IntegerType, true),
    StructField("FireplaceQu", StringType, true),
    StructField("GarageType", StringType, true),
    StructField("GarageYrBlt", StringType, true),
    StructField("GarageFinish", StringType, true),
    StructField("GarageCars", StringType, true),
    StructField("GarageArea", StringType, true),
    StructField("GarageQual", StringType, true),
    StructField("GarageCond", StringType, true),
    StructField("PavedDrive", StringType, true),
    StructField("WoodDeckSF", IntegerType, true),
    StructField("OpenPorchSF", IntegerType, true),
    StructField("EnclosedPorch", IntegerType, true),
    StructField("3SsnPorch", IntegerType, true),    
    StructField("ScreenPorch", IntegerType, true),
    StructField("PoolArea", IntegerType, true),
    StructField("PoolQC", StringType, true),
    StructField("Fence", StringType, true),
    StructField("MiscFeature", StringType, true),
    StructField("MiscVal", IntegerType, true),
    StructField("MoSold", IntegerType, true),
    StructField("YrSold", IntegerType, true),
    StructField("SaleType", StringType, true),
    StructField("SaleCondition", StringType, true),
    StructField("SalePrice", IntegerType, true)
  ))

In [10]:
val train = spark.read.option("header", true).schema(schema).csv("train.csv").cache()
val test = spark.read.option("header", true).schema(schema).csv("test.csv").cache()

### Example of using Spark SQL

In [126]:
train.createOrReplaceTempView("train")

In [127]:
%%sql 
SELECT neighborhood, avg(saleprice) as averageNeighborhoodPrice FROM train GROUP BY neighborhood ORDER BY averageNeighborhoodPrice DESC

+------------+------------------------+
|neighborhood|averageNeighborhoodPrice|
+------------+------------------------+
|     NoRidge|      335295.31707317074|
|     NridgHt|       316270.6233766234|
|     StoneBr|                310499.0|
|      Timber|      242247.44736842104|
|     Veenker|      238772.72727272726|
|     Somerst|      225379.83720930232|
|     ClearCr|      212565.42857142858|
|     Crawfor|      210624.72549019608|
|     CollgCr|      197965.77333333335|
|     Blmngtn|      194870.88235294117|
+------------+------------------------+
only showing top 10 rows



In [128]:
val vis = spark.sql("SELECT neighborhood, count(id) as count FROM train GROUP BY neighborhood ORDER BY count DESC")

In [129]:
%%brunel
data('vis') bar color(neighborhood) y(count) polar stack label(neighborhood)

## Local Split for Cross-validation

In [11]:
val split = train.randomSplit(Array(0.6, 0.4))
val localTrain = split(0)
val localTest = split(1)

## Data Preparation

### Convert categorical values to numerical

In [12]:
val categorical = Array("MSZoning","LotFrontage","Street","Alley","LotShape","LandContour","Utilities","LotConfig",
"LandSlope","Neighborhood","Condition1","Condition2","BldgType","HouseStyle","RoofStyle","RoofMatl","Exterior1st",
"Exterior2nd","MasVnrType","MasVnrArea","ExterQual","ExterCond","Foundation","BsmtQual","BsmtCond","BsmtExposure",
"BsmtFinType1","BsmtFinType2","Heating","HeatingQC","CentralAir","Electrical","KitchenQual","Functional",
"FireplaceQu","GarageType","GarageYrBlt","GarageFinish","GarageQual","GarageCond","PavedDrive","PoolQC","Fence",
"MiscFeature","SaleType","SaleCondition","BsmtFinSF1","BsmtFinSF2","BsmtUnfSF","TotalBsmtSF","BsmtFullBath",
"BsmtHalfBath", "GarageCars","GarageArea","MiscVal")

val numerical = Array("MSSubClass","LotArea","OverallQual","OverallCond","YearBuilt","YearRemodAdd",
"1stFlrSF","2ndFlrSF","LowQualFinSF","GrLivArea","FullBath","HalfBath","BedroomAbvGr","KitchenAbvGr","TotRmsAbvGrd",
"Fireplaces","WoodDeckSF","OpenPorchSF","EnclosedPorch","3SsnPorch","ScreenPorch","PoolArea","MoSold","YrSold")

In [13]:
val stringIndexers = (for {
    f <- categorical
} yield {
    new StringIndexer()  
      .setInputCol(f)
      .setOutputCol(s"${f}_idx")
      .setHandleInvalid("skip")
}).toArray

In [14]:
val categoricalIdx = categorical.map(f => s"${f}_idx")

In [15]:
val pipelineIdx = { new Pipeline()  
  .setStages(stringIndexers)
}

val modelIdx = pipelineIdx.fit(localTrain)  
val trainIdx = modelIdx.transform(localTrain).cache()

In [16]:
trainIdx.select("SalePrice","Neighborhood","Neighborhood_idx","SaleType","SaleType_idx","YearRemodAdd").show()

+---------+------------+----------------+--------+------------+------------+
|SalePrice|Neighborhood|Neighborhood_idx|SaleType|SaleType_idx|YearRemodAdd|
+---------+------------+----------------+--------+------------+------------+
|   208500|     CollgCr|             1.0|      WD|         0.0|        2003|
|   181500|     Veenker|            23.0|      WD|         0.0|        1976|
|   223500|     CollgCr|             1.0|      WD|         0.0|        2002|
|   250000|     NoRidge|            15.0|      WD|         0.0|        2000|
|   143000|     Mitchel|            11.0|      WD|         0.0|        1995|
|   129900|     OldTown|             2.0|      WD|         0.0|        1950|
|   118000|     BrkSide|             9.0|      WD|         0.0|        1950|
|   129500|      Sawyer|             7.0|      WD|         0.0|        1965|
|   345000|     NridgHt|             8.0|     New|         1.0|        2006|
|   144000|      Sawyer|             7.0|      WD|         0.0|        1962|

### Assemble All the Columns of Interest

In [17]:
val assembler = { new VectorAssembler()  
  .setInputCols((numerical ++ categoricalIdx).toArray)
  .setOutputCol("features")
}

# Unsupervised Methods and Visualisation

## PCA

In [18]:
val pca = { new PCA()
    .setInputCol("features")
    .setOutputCol("pca")
    .setK(2)
}

In [140]:
val pipelinePCA = { new Pipeline()  
  .setStages(stringIndexers ++ Array(assembler, pca))
}

val modelPCA = pipelinePCA.fit(localTrain)  
val resultPCA = modelPCA.transform(localTrain).cache()

In [141]:
resultPCA.select("id","features","pca").show(2)

+---+--------------------+--------------------+
| id|            features|                 pca|
+---+--------------------+--------------------+
|  2|(79,[0,1,2,3,4,5,...|[9680.01916022798...|
|  6|(79,[0,1,2,3,4,5,...|[14181.7587737500...|
+---+--------------------+--------------------+
only showing top 2 rows



### Example of User Defined Functions is Spark

In [142]:
val xUdf = udf((a: Vector) => a(0))
val yUdf = udf((a: Vector) => a(1))

In [143]:
val visPCA = resultPCA.withColumn("x", xUdf(resultPCA("pca"))).withColumn("y",yUdf(resultPCA("pca")))

In [144]:
visPCA.select("saleprice","x","y").show()

+---------+------------------+-------------------+
|saleprice|                 x|                  y|
+---------+------------------+-------------------+
|   181500| 9680.019160227983| -878.6977962199336|
|   143000|14181.758773750027|-1012.4426952038934|
|   307000| 10195.21169645103|  -1288.75420007482|
|   200000|10495.660197733496|-2046.2796272715552|
|   129500|11260.842771391806| -583.9683495323557|
|   279500|10748.483695376483|-1080.9419426705022|
|   132000|6173.7489780673795| -616.6635950405982|
|   159000|13758.550186292885| -568.0147129099967|
|   325300|14339.553209857084| -2276.893587291321|
|   139400| 7520.610718602387| -820.7469798455576|
|   129900| 4295.881073026086| -902.8135977987241|
|   154000| 8316.284748898588| -765.0147598486856|
|   306000|11586.014984751911|-1230.0836106224672|
|   207500|16419.191080334775| -956.4850163436628|
|   149350| 8623.088322139381| -896.2994754307116|
|   165500|10662.701727706173| -1275.360961506033|
|   277500| 7418.147932007273|-

In [145]:
%%brunel
data('visPCA') x(x) y(y) color(saleprice)

In [146]:
%%brunel
data('visPCA') x(x) y(y) label(Neighborhood_idx) color(saleprice)

## K-Means

In [147]:
val kMeans = { new KMeans()
    .setFeaturesCol("features")
    .setPredictionCol("kmeans")
    .setK(500) 
}

In [148]:
val pipelineKM = { new Pipeline()  
  .setStages(stringIndexers ++ Array(assembler, kMeans))
}

val modelKM = pipelineKM.fit(localTrain)  
val resultKM = modelKM.transform(localTrain).cache()

In [149]:
resultKM.select("features").take(1).foreach(println)

[(79,[0,1,2,3,4,5,6,9,10,12,13,14,15,16,22,23,25,31,33,34,40,41,46,47,49,50,58,60,61,70,72,73,75,77],[20.0,9600.0,6.0,8.0,1976.0,1976.0,1262.0,1262.0,2.0,3.0,1.0,6.0,1.0,298.0,5.0,2007.0,4.0,3.0,22.0,1.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,14.0,1.0,266.0,63.0,62.0,1.0,36.0])]


In [150]:
val visKM = resultKM.select("SalePrice", "Neighborhood_idx", "kmeans")

In [151]:
visKM.show(10)

+---------+----------------+------+
|SalePrice|Neighborhood_idx|kmeans|
+---------+----------------+------+
|   181500|            22.0|   102|
|   143000|             8.0|   327|
|   307000|             7.0|   147|
|   200000|             9.0|   170|
|   129500|             6.0|   217|
|   279500|             1.0|   496|
|   132000|            10.0|   296|
|   159000|            11.0|   426|
|   325300|             5.0|   360|
|   139400|            13.0|    87|
+---------+----------------+------+
only showing top 10 rows



In [152]:
%%brunel
data('visKM') x(saleprice) y(kmeans) bin(saleprice) color(saleprice) 

## Model buildiing and evaluation

### Simple Linear Regression

In [19]:
val lr = new LinearRegression().
            setFeaturesCol("features").
            setPredictionCol("predictions").
            setLabelCol("SalePrice").
            setMaxIter(10).
            setRegParam(0.3).
            setElasticNetParam(0.8)

In [20]:
val pipelineLR = { new Pipeline()  
  .setStages(stringIndexers ++ Array(assembler, lr))
}

val modelLR = pipelineLR.fit(localTrain)  
val resultLR = modelLR.transform(localTest)

In [112]:
localTest.count

584

In [113]:
resultLR.count

106

In [28]:
val m = modelLR.stages(stringIndexers.length+1).asInstanceOf[LinearRegressionModel]
println(s"Intercept: ${m.intercept}\nCoefficients: ${m.coefficients}")

Intercept: 603493.6379922332
Coefficients: [-168.6519152279232,0.3496634678313195,14088.907521555722,4035.0259874438557,185.48454182654402,146.3900387821531,23.21965640428692,17.85825252375195,-15.112201521482284,22.046220811265304,10354.72447731076,6517.178346447138,-5792.0639203466935,-2775.741048237291,2724.6450398400434,3961.4986005365454,23.76010295257767,11.169364498766887,-11.81246787765207,66.8655420765533,63.16959546435761,-112.08430515072934,-446.5258224674097,-648.6187755737701,-1636.6690685093586,147.72948545704241,-43071.72116665344,-1418.7014535257538,-0.0,-1997.5031337704795,-37923.46025871694,613.8766141503419,12729.063654032461,619.8181446868003,-2465.9758086305405,-34344.3463020006,-928.9493277578733,-604.6504800264302,2215.1603634261755,-15403.132685228782,-714.4869693022381,-142.89569333515254,891.8594258625222,-2.802108424853231,11812.766383976093,-965.7097285933359,-4878.796718669985,8901.632720674306,-7577.52132773562,5006.14801638234,-345.5843897953969,0.0,4353.

In [22]:
val evaluator = { new RegressionEvaluator()  
  .setLabelCol("SalePrice")
  .setPredictionCol("predictions")
  .setMetricName("rmse")
}

In [23]:
println(s"Root Mean Squared Error (RMSE) on test data = ${evaluator.evaluate(resultLR)}")

Root Mean Squared Error (RMSE) on test data = 25410.188021656824


#### Kaggle uses RMSE on log

In [36]:
val logUdf = udf((a: Double) => math.log(a))

In [79]:
val resultLRLog = resultLR.withColumn("logSalePrice", logUdf(resultLR("SalePrice"))).
    withColumn("logPredictions", logUdf(resultLR("predictions"))).
    select("logSalePrice","logPredictions").orderBy(asc("logSalePrice"))

In [81]:
val evaluator = { new RegressionEvaluator()  
  .setLabelCol("logSalePrice")
  .setPredictionCol("logPredictions")
  .setMetricName("rmse")
}

In [82]:
println(s"RMSE of logs on test data = ${evaluator.evaluate(resultLRLog)}")

RMSE of logs on test data = 0.20836723326174802


In [83]:
resultLRLog.show()

+------------------+------------------+
|      logSalePrice|    logPredictions|
+------------------+------------------+
|10.542706391070517|11.315878907347411|
|10.578979797857352|11.186533944728575|
|11.133128103610641|11.180736988563426|
| 11.27720313144916|11.694343906859936|
|11.288531131754366|11.728176990013223|
|11.289781913656018|11.398508120748051|
|11.314474526246391| 10.64175286297382|
|11.350406535472453|11.694295607415723|
|11.356271654924852|11.599508021494167|
|11.379394072345706|11.167015842487007|
|11.439278924801929| 11.13645211869843|
|11.440354772135393|11.580965319184733|
| 11.44571671527678|11.098049557942325|
|11.451050061252142|11.579246534954539|
|11.477298287327077|11.512890159945801|
| 11.48246625748552|11.672608417656313|
| 11.48246625748552|11.609123613776601|
| 11.49272275765271|11.468394368806905|
|11.495779306135258| 11.53616304516025|
|11.511924964636645|11.877322799949894|
+------------------+------------------+
only showing top 20 rows



In [84]:
%%brunel
data('resultLRLog') x(#row) y(logSalePrice,logPredictions) color(#series) line

### Random Forest

In [52]:
val featureIndexer = new VectorIndexer().
    setInputCol("features").
    setOutputCol("indexedFeatures").
    setMaxCategories(16)
    
    
    isn't working or what?

In [60]:
val rf = new RandomForestRegressor().
    setFeaturesCol("features").
    setPredictionCol("predictions").
    setLabelCol("SalePrice").
    setNumTrees(10).
    setMaxDepth(10).
    setSeed(42L).
    setMaxBins(600)

In [61]:
val pipelineRF = { new Pipeline()  
  //.setStages(stringIndexers ++ Array(assembler, featureIndexer, rf))
  .setStages(stringIndexers ++ Array(assembler, rf))
}

In [62]:
val modelRF = pipelineRF.fit(localTrain)  
val resultRF = modelRF.transform(localTest).cache()

In [85]:
val rfModel = modelRF.stages(stringIndexers.length+1).asInstanceOf[RandomForestRegressionModel]
println(rfModel.toDebugString)

RandomForestRegressionModel (uid=rfr_e3a739d3c87d) with 10 trees
  Tree 0 (weight 1.0):
    If (feature 77 in {0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0,30.0,31.0,32.0,33.0,34.0,35.0,36.0,37.0,38.0,39.0,40.0,41.0,43.0,44.0,45.0,46.0,47.0,48.0,50.0,51.0,52.0,53.0,54.0,55.0,56.0,57.0,58.0,60.0,62.0,64.0,65.0,66.0,67.0,69.0,70.0,72.0,74.0,75.0,76.0,77.0,78.0,79.0,80.0,82.0,84.0,85.0,86.0,87.0,88.0,89.0,91.0,93.0,94.0,96.0,98.0,100.0,101.0,102.0,104.0,106.0,107.0,109.0,111.0,113.0,116.0,117.0,118.0,120.0,121.0,122.0,123.0,124.0,127.0,129.0,131.0,134.0,135.0,136.0,137.0,138.0,139.0,140.0,141.0,143.0,145.0,146.0,147.0,148.0,149.0,155.0,161.0,164.0,166.0,167.0,168.0,171.0,175.0,177.0,180.0,182.0,185.0,186.0,187.0,189.0,190.0,193.0,194.0,195.0,197.0,200.0,201.0,202.0,204.0,206.0,207.0,208.0,210.0,213.0,220.0,223.0,224.0,226.0,227.0,229.0,230.0,232.0,233.0,234.0,235.0,243.0,245.0,250.0,251.0,253.0,

In [89]:
val resultRFLog = resultRF.withColumn("logSalePrice", logUdf(resultRF("SalePrice"))).
    withColumn("logPredictions", logUdf(resultRF("predictions"))).
    select("logSalePrice","logPredictions").orderBy(asc("logSalePrice"))

In [90]:
resultRFLog.show()

+------------------+------------------+
|      logSalePrice|    logPredictions|
+------------------+------------------+
|10.542706391070517|  11.9245023750451|
|10.578979797857352|11.397703186565913|
|11.133128103610641|12.583501578710026|
| 11.27720313144916|11.849098322366368|
|11.288531131754366|12.025297036000861|
|11.289781913656018|11.880712294452495|
|11.314474526246391|11.641065214608208|
|11.350406535472453|11.451718530785708|
|11.356271654924852|12.114142015276322|
|11.379394072345706|11.743382879703905|
|11.439278924801929|11.661925487798785|
|11.440354772135393|11.467836130480274|
| 11.44571671527678|11.696781269597082|
|11.451050061252142|11.794707838060816|
|11.477298287327077| 11.80558369714055|
| 11.48246625748552|12.115009900395256|
| 11.48246625748552|11.410631140637964|
| 11.49272275765271| 11.83522244324304|
|11.495779306135258|11.584608497706865|
|11.511924964636645|11.612950791641062|
+------------------+------------------+
only showing top 20 rows



In [91]:
println(s"RMSE of logs on test data = ${evaluator.evaluate(resultRFLog)}")

RMSE of logs on test data = 0.5624093571308767


In [92]:
%%brunel
data('resultRFLog') x(#row) y(logSalePrice,logPredictions) color(#series) line

## Full model 

In [100]:
val model = pipelineLR.fit(train)  
val result = modelLR.transform(test).cache()

In [102]:
result.select("id","predictions").show

+----+------------------+
|  Id|         SalePrice|
+----+------------------+
|1482| 219498.9063370807|
|1484| 184026.3155270532|
|1486|203759.99002014817|
|1492| 91114.25987269531|
|1497|176945.97508988465|
|1498| 162468.9467875918|
|1513| 148946.7966289018|
|1527| 90272.13429983507|
|1538|201391.04644470062|
|1540|167121.17758731532|
|1546|149319.92574338068|
|1548|134611.55918543326|
|1550|123672.79171354644|
|1553|187629.52582336235|
|1554|135864.42013652495|
|1567| 83092.20466636331|
|1578|166054.93667912943|
|1585|144058.22598608886|
|1586| 88647.37004713505|
|1589|105026.64277430775|
+----+------------------+
only showing top 20 rows



In [3]:
lets see how our model is doing on the leader board

5

In [111]:
result.count

215

In [110]:
test.count

1459

In [107]:
result.select("id","predictions").toDF("Id","SalePrice").
coalesce(1).
write.option("header",true).csv("demo-predicitons.csv")

## Streaming